This is a live notebook with experimental code to develop analysis investigating the correlation between features.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import json

import scipy.stats as spstats
# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.ar_model import ar_select_order

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'


['debug_windowing_data.ipynb', 'testing_mlpwrapper.ipynb', 'logs', 'analyse_results.ipynb', 'gait_data_features.ipynb', 'feature_development.ipynb', 'data', 'feature_analysis.ipynb', 'merging_dataframes.ipynb', 'data_and_processing_description.ipynb', 'featured_prediction_random_forest.ipynb', 'window_timings_and_IBI.ipynb', 'gait_data_exploration.ipynb', 'processing_status.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'data_investigation_scratch.ipynb', 'simply_load_and_inspect_data.ipynb', 'grouping_and_crossvalidation.ipynb', 'sample-entropy-numba-impl.ipynb', 'featured_prediction.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH
from predicament.utils.file_utils import drop_nan_cols
from predicament.utils.file_utils import load_windowed_data_and_config
from predicament.utils.file_utils import load_dataframe_and_config
from predicament.utils.file_utils import write_dataframe_and_config

from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from predicament.data.features import filter_features

from prepare_evaluation_data import group_conditions

In [5]:
from predicament.utils.config import E4_CSV_FILES
from predicament.utils.config import E4_FULL_DIRPATHS
E4_FULL_DIRPATHS

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

In [6]:
subdir = 'binary_dreem_4secs'
df, config = load_windowed_data_and_config(subdir)
df

Reading dataframe from ../data/windowed/binary_dreem_4secs/windowed.csv
Reading config from ../data/windowed/binary_dreem_4secs/details.cfg


,participant,condition,window index,EEG Fpz-O1[0],EEG Fpz-O1[1],EEG Fpz-O1[2],EEG Fpz-O1[3],EEG Fpz-O1[4],EEG Fpz-O1[5],EEG Fpz-O1[6],...,EEG Fpz-F8[1014],EEG Fpz-F8[1015],EEG Fpz-F8[1016],EEG Fpz-F8[1017],EEG Fpz-F8[1018],EEG Fpz-F8[1019],EEG Fpz-F8[1020],EEG Fpz-F8[1021],EEG Fpz-F8[1022],EEG Fpz-F8[1023]
0,VG_01,0,0,0.000012,1.496147e-05,0.000011,0.000003,-0.000006,-0.000015,-0.000019,...,-1.119249e-05,-0.000009,-6.691081e-06,-0.000005,-0.000005,-6.645304e-06,-1.006332e-05,-1.253529e-05,-0.000013,-1.326772e-05
1,VG_01,0,1,-0.000007,-6.996262e-06,-0.000014,-0.000027,-0.000043,-0.000055,-0.000057,...,2.437629e-05,0.000014,1.945525e-06,-0.000007,-0.000008,5.264363e-07,1.168078e-05,1.465629e-05,0.000005,-7.362478e-06
2,VG_01,0,2,0.000029,2.675669e-05,0.000027,0.000033,0.000042,0.000047,0.000043,...,-8.163577e-07,-0.000002,9.689479e-07,0.000005,0.000005,6.485084e-07,-3.822385e-06,-5.271992e-06,-0.000004,-4.043641e-07
3,VG_01,0,3,-0.000002,2.975509e-07,0.000001,-0.000004,-0.000011,-0.000014,-0.000010,...,8.812085e-06,0.000015,1.064317e-05,0.000001,-0.000003,-2.357519e-06,-9.994659e-07,-3.318837e-06,-0.000007,-9.178302e-06
4,VG_01,0,4,-0.000058,-5.281910e-05,-0.000040,-0.000027,-0.000022,-0.000025,-0.000032,...,2.814527e-05,0.000039,4.871443e-05,0.000052,0.000043,2.555123e-05,8.232242e-06,8.392462e-08,0.000003,1.274891e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5622,VG_09,0,456,-0.000011,-1.066606e-05,-0.000008,-0.000007,-0.000011,-0.000018,-0.000026,...,7.104601e-06,0.000008,1.002976e-05,0.000012,0.000012,8.065919e-06,1.831083e-06,-2.645914e-06,-0.000002,4.673838e-06
5623,VG_09,0,457,0.000007,6.981003e-06,0.000008,0.000011,0.000018,0.000027,0.000037,...,-1.120165e-05,-0.000010,-6.285191e-06,-0.000002,0.000003,6.623941e-06,8.271916e-06,7.859922e-06,0.000007,6.857404e-06
5624,VG_09,0,458,0.000001,8.601511e-06,0.000016,0.000019,0.000017,0.000012,0.000009,...,1.203479e-05,0.000008,2.764935e-06,-0.000003,-0.000005,-4.898146e-06,-1.203937e-06,4.357977e-06,0.000009,1.162280e-05
5625,VG_09,0,459,0.000029,2.888533e-05,0.000030,0.000032,0.000030,0.000024,0.000018,...,1.954681e-06,0.000002,3.698787e-06,0.000007,0.000009,1.178759e-05,1.384756e-05,1.421836e-05,0.000011,5.552758e-06


In [7]:
config

{'DEFAULT': {},
 'LOAD': {'participant_list': ['VG_01', 'VG_09'],
  'conditions': ['baseline',
   'break',
   'exper_video',
   'wildlife_video',
   'familiar_music',
   'tchaikovsky',
   'family_inter'],
  'channels': ['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG Fpz-F8'],
  'n_channels': 4,
  'sample_rate': 250,
  'window_size': 1024,
  'window_step': 128,
  'label_mapping': ['active', 'inactive'],
  'data_format': 'dreem',
  'label_groups': {'inactive': ['baseline', 'break'],
   'active': ['exper_video',
    'wildlife_video',
    'familiar_music',
    'tchaikovsky',
    'family_inter']}},
 'WINDOWED': {'group_col': 'participant',
  'target_col': 'condition',
  'label_cols': ['participant', 'condition', 'window index']}}

In [8]:
df['condition'].unique()



array([0, 1])

In [ ]:
string_ = "inactive:baseline,break;active:exper_video,wildlife_video,familiar_music,tchaikovsky,family_inter"
dict_ = get_dict_from_string(string_)
dict_

In [ ]:
get_dict_of_lists_from_dict_of_strings(dict_)